In [2]:
from bs4 import BeautifulSoup, Comment
import requests as req
from selenium import webdriver
import time

import re
import csv
import pandas as pd

In [3]:
link = 'https://en.art-most.com/all_pictures/page/'

FILENAME = 'C:/Users/Andrew/Desktop/ML/parsing/gallery.csv'

In [5]:
def get_data(links):
    '''
    Get all necessary data from a page and write to file
    Arguments:
      links - link to a specific page

    Returns:
      Write to the file information from a page
    '''
    soup_child = BeautifulSoup(resp_child.text.encode(encoding='utf-8', errors='ignore').decode('utf-8', 'ignore'), 'lxml')
    info = soup_child.find('div', class_ = 'wrapper').find('div', class_ = 'row').find('div', class_ = 'row')
    
#   here we get actual necessary info
    author = info.find('div', class_ = 'row artist').text.strip()
    name = info.find('h1', class_ = 'product_title entry-title').text
#   cut symbol € and prepare to int
    price = info.find('p', class_ = 'price').text[:-2].replace(',', '')

    part = info.find(id = 'art_div_original_2').find_all('p')
    material = part[0].text[10:].replace('"', '')
    technic = part[1].text[10:].replace('"', '')
    if (part[2].text[6:].replace('"', '').replace(' ', '') == 'Unknown'):
        vertical_size = 'Unknown'
        horizontal_size = 'Unknown'
    else:
        vertical_size = part[2].text[6:].split(' ')[0].replace(',', '.').replace('"', '')
        horizontal_size = part[2].text[6:].split(' ')[2].replace(',', '.').replace('"', '')
    year = part[3].text[6:]
    genre = part[4].text[7:]
    style = part[5].text[7:]
    
#   create a dictionary and write to a file

    dict = {'author': author, 'name': name, 'material': material, 'technic': technic, 'genre': genre, 'style': style, 
        'v_size': vertical_size, 'h_size': horizontal_size, 'year': year, 'price': price}

    with open(FILENAME, 'a', newline='', encoding='utf-8') as file:
        columns = ['author', 'name', 'material', 'technic', 'genre', 'style', 'v_size', 'h_size', 'year', 'price']
        writer = csv.DictWriter(file, fieldnames=columns)
#        writer.writeheader()
        writer.writerow(dict)
        file.close()
        
    time.sleep(2)

In [19]:
#get information from the first page

resp = req.get('https://en.art-most.com/all_pictures/')
soup = BeautifulSoup(resp.text, 'lxml')
page = soup.find('ul', class_= 'products columns-3')
for heading in page.find_all('a', class_='woocommerce-LoopProduct-link woocommerce-loop-product__link'):
    resp_child = req.get(heading['href'])
    get_data(resp_child)

In [6]:
#get information from all next pages

for i in range(2, 381):
    link_to_page = link + str(i) + '/'
    resp = req.get(link_to_page)
    soup = BeautifulSoup(resp.text, 'lxml')
    page = soup.find('ul', class_= 'products columns-3')
    for heading in page.find_all('a', class_='woocommerce-LoopProduct-link woocommerce-loop-product__link'):
        resp_child = req.get(heading['href'])
        get_data(resp_child)